In [82]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from scripts.word_embeddings import load_embedding_weights

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

In [83]:
def save(name, words, vectors, path='.'):
    with open(f'{path}/{name}.txt', 'w+', encoding='utf-8') as doc:
        for word, vector in zip(words, vectors):
            doc.write(word + ' ' + ' '.join(str(value) for value in vector))
            doc.write('\n')

# Load Dataset

In [84]:
df = pd.read_csv('data/trial.csv',delimiter="	")
df.columns = ['file_name', 'misogynous', 'shaming', 'stereotype', 'objectification', 'violence', 'text_transcription']
df.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,text_transcription
0,28.jpg,0,0,0,0,0,"not now, dad. We should burn Jon Snow. stop it..."
1,30.jpg,0,0,0,0,0,there may have been a mixcommunication with th...
2,33.jpg,0,0,0,0,0,i shouldn't have sold my boat
3,58.jpg,1,0,0,0,1,"Bitches be like, It was my fault i made him mad"
4,89.jpg,0,0,0,0,0,find a picture of 4 girls together on FB make ...


# Create Embeddings

In [85]:
from nltk import word_tokenize
from nltk.corpus import stopwords
ENGLISH_STOPWORDS = stopwords.words('english')

def remove_punctuation(token:str)->str:
    punctuation_regex = '!"#$&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    return ' '.join(word.strip(punctuation_regex) for word in token.split())

def nlp_pipeline(token:str) -> str:
    token = remove_punctuation(token)
    tokens = word_tokenize(token.lower())
    tokens = [token for token in tokens if token not in ENGLISH_STOPWORDS]
    return tokens

df['text_transcription_tokens'] = df['text_transcription'].apply(lambda x: nlp_pipeline(x))
df.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,text_transcription,text_transcription_tokens
0,28.jpg,0,0,0,0,0,"not now, dad. We should burn Jon Snow. stop it...","[dad, burn, jon, snow, stop, dad, know, happen..."
1,30.jpg,0,0,0,0,0,there may have been a mixcommunication with th...,"[may, mixcommunication, decorator, happy, birt..."
2,33.jpg,0,0,0,0,0,i shouldn't have sold my boat,"[n't, sold, boat]"
3,58.jpg,1,0,0,0,1,"Bitches be like, It was my fault i made him mad","[bitches, like, fault, made, mad]"
4,89.jpg,0,0,0,0,0,find a picture of 4 girls together on FB make ...,"[find, picture, 4, girls, together, fb, make, ..."


## Word2Vec

In [86]:
sentences = df['text_transcription_tokens'].values

def build_embeddings(model_name, sentences, size, path):
    if model_name == 'word2vec':
        model = Word2Vec(sentences, vector_size=size, min_count=1, window=5, sg=1)
        vectors = model.wv.vectors      
        words = model.wv.index_to_key

        save(f'{model_name}SG.iSarcasamEval.{size}d', words, vectors,path=path)

for size in [10, 50, 100]:
    build_embeddings('word2vec', sentences, size=size, path='.')

# Task 1

## Tokenization and vocabulary creation

In [87]:
sentences = df['text_transcription_tokens'].values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [88]:
vocabulary = list(set(tokenizer.word_index.keys()))
vocabulary_size = len(tokenizer.word_index)
max_length = max(map(lambda x: len(x), tokenizer.word_index.keys()))

X = tokenizer.texts_to_sequences(sentences)
X_pad = pad_sequences(X, maxlen=max_length, padding='post')

In [89]:
y = df['misogynous']

## Load pre-trained weights

In [111]:
embeddings = {
    10: load_embedding_weights(vocabulary, 10, 'word2vecSG', "."),
    50: load_embedding_weights(vocabulary, 50, 'word2vecSG', "."),
    100: load_embedding_weights(vocabulary, 100, 'word2vecSG', ".")
}

In [114]:
glove_50 = load_embedding_weights(vocabulary, 50, 'glove', "/mnt/d/Downloads")

Creating embedding weights...


## Define Models

In [112]:
import wandb
from wandb.keras import WandbCallback

In [92]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.1)

### Without initialized weights

In [109]:
model = Sequential(name="withoutInitializedWeights")
model.add(Embedding(input_dim = vocabulary_size + 1, output_dim=100))
model.add(LSTM(units=100, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate = 0.01), loss=binary_crossentropy, metrics=['accuracy'])
model.summary()

Model: "withoutInitializedWeights"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_27 (Embedding)    (None, None, 100)         47100     
                                                                 
 lstm_9 (LSTM)               (None, None, 100)         80400     
                                                                 
 dense_22 (Dense)            (None, None, 1)           101       
                                                                 
Total params: 127,601
Trainable params: 127,601
Non-trainable params: 0
_________________________________________________________________


In [110]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=100, callbacks=[WandbCallback()])
run.finish()

2021-11-13 17:32:23.992145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-13 17:32:23.992188: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/100
3/3 [==============================] - 2s 52ms/step - loss: 0.6933 - accuracy: 0.5090
Epoch 2/100
3/3 [==============================] - 0s 33ms/step - loss: 0.6932 - accuracy: 0.5919
Epoch 3/100
3/3 [==============================] - 0s 34ms/step - loss: 0.6652 - accuracy: 0.6584
Epoch 4/100
3/3 [==============================] - 0s 31ms/step - loss: 0.3178 - accuracy: 0.9442
Epoch 5/100
3/3 [==============================] - 0s 30ms/step - loss: 0.2090 - accuracy: 0.9625
Epoch 6/100
3/3 [==============================] - 0s 34ms/step - loss: 0.0677 - accuracy: 0.9845
Epoch 7/100
3/3 [==============================] - 0s 31ms/step - loss: 0.0244 - accuracy: 0.9959
Epoch 8/100
3/3 [==============================] - 0s 35ms/step - loss: 0.0237 - accuracy: 0.9966
Epoch 9/100
3/3 [==============================] - 0s 32ms/step - loss: 0.0186 - accuracy: 0.9976
Epoch 10/100
3/3 [==============================] - 0s 31ms/step - loss: 0.0134 - accuracy: 0.9976
Epoch 11/100
3/3 [=

accuracy,▁▃██████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.99759
epoch,99
loss,0.00316


### Word2Vec 50

In [104]:
model = Sequential(name="word2vec.50d")

model.add(Embedding(input_dim = vocabulary_size + 1, weights=[embeddings[100]], output_dim=100))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.01), loss=binary_crossentropy, metrics=['accuracy'])
model.summary()

Model: "word2vec.50d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, None, 100)         47100     
                                                                 
 lstm_6 (LSTM)               (None, None, 64)          42240     
                                                                 
 lstm_7 (LSTM)               (None, None, 128)         98816     
                                                                 
 dense_20 (Dense)            (None, None, 1)           129       
                                                                 
Total params: 188,285
Trainable params: 188,285
Non-trainable params: 0
_________________________________________________________________


In [106]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=50, callbacks=[WandbCallback()])
run.finish()

2021-11-13 17:31:28.069684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-13 17:31:28.069729: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/50
3/3 [==============================] - 0s 78ms/step - loss: 0.0033 - accuracy: 0.9976
Epoch 2/50
3/3 [==============================] - 0s 70ms/step - loss: 0.0033 - accuracy: 0.9983
Epoch 3/50
3/3 [==============================] - 0s 61ms/step - loss: 0.0032 - accuracy: 0.9983
Epoch 4/50
3/3 [==============================] - 0s 57ms/step - loss: 0.0033 - accuracy: 0.9979
Epoch 5/50
3/3 [==============================] - 0s 63ms/step - loss: 0.0032 - accuracy: 0.9983
Epoch 6/50
3/3 [==============================] - 0s 59ms/step - loss: 0.0032 - accuracy: 0.9983
Epoch 7/50
3/3 [==============================] - 0s 60ms/step - loss: 0.0032 - accuracy: 0.9983
Epoch 8/50
3/3 [==============================] - 0s 64ms/step - loss: 0.0032 - accuracy: 0.9983
Epoch 9/50
3/3 [==============================] - 0s 57ms/step - loss: 0.0032 - accuracy: 0.9983
Epoch 10/50
3/3 [==============================] - 0s 63ms/step - loss: 0.0032 - accuracy: 0.9979
Epoch 11/50
3/3 [============

accuracy,▁██▅████████▅█████████████▄█▅██▅██▅███▅█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇█▆▆▆▆▆▆▆▆▅▅▅▄▄▄▄▃▄▄▄▃▃▂▄▄▃▂▁▂▂▂▂▂▁▁▂▁
accuracy,0.99828
epoch,49
loss,0.00301


### GloVe 50

In [124]:
model = Sequential(name="glove.50d")

model.add(Embedding(input_dim = vocabulary_size+1, output_dim=50))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.05), loss=binary_crossentropy, metrics=['accuracy'])
model.summary()

Model: "glove.50d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_33 (Embedding)    (None, None, 50)          23550     
                                                                 
 lstm_14 (LSTM)              (None, None, 128)         91648     
                                                                 
 dense_26 (Dense)            (None, None, 1)           129       
                                                                 
Total params: 115,327
Trainable params: 115,327
Non-trainable params: 0
_________________________________________________________________


In [126]:
run = wandb.init(reinit=True, name=model.name)
model.fit(X_train, y_train, epochs=50, callbacks=[WandbCallback()])
run.finish()

2021-11-13 18:01:33.792602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-13 18:01:33.792646: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/50
3/3 [==============================] - 0s 46ms/step - loss: 0.0040 - accuracy: 0.9983
Epoch 2/50
3/3 [==============================] - 0s 44ms/step - loss: 0.0040 - accuracy: 0.9983
Epoch 3/50
3/3 [==============================] - 0s 38ms/step - loss: 0.0039 - accuracy: 0.9983
Epoch 4/50
3/3 [==============================] - 0s 37ms/step - loss: 0.0039 - accuracy: 0.9983
Epoch 5/50
3/3 [==============================] - 0s 34ms/step - loss: 0.0038 - accuracy: 0.9983
Epoch 6/50
3/3 [==============================] - 0s 36ms/step - loss: 0.0038 - accuracy: 0.9983
Epoch 7/50
3/3 [==============================] - 0s 38ms/step - loss: 0.0037 - accuracy: 0.9983
Epoch 8/50
3/3 [==============================] - 0s 37ms/step - loss: 0.0037 - accuracy: 0.9983
Epoch 9/50
3/3 [==============================] - 0s 36ms/step - loss: 0.0037 - accuracy: 0.9983
Epoch 10/50
3/3 [==============================] - 0s 33ms/step - loss: 0.0036 - accuracy: 0.9983
Epoch 11/50
3/3 [============

accuracy,████████████▁█▄█▄▄██▅███▁▄▅▅██████████▄█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
accuracy,0.99828
epoch,49
loss,0.00303


WANDB was used to track the runs, all of the results are available at the [link](https://wandb.ai/aleksandar1932/NLP_2021-Laboratory%20Exercises_2?workspace=user-aleksandar1932).